In [1]:
import pyspark
import math
import matplotlib.pyplot as plt
import numpy as np
from pyspark import SparkContext
from os import listdir
from os.path import isfile, join
from pyspark.mllib.feature import HashingTF, IDF
from pyspark.mllib.clustering import KMeans

%matplotlib inline

mypath = "../../project/cmsc25025/sou/text"  #Working directory

In [2]:
# Get a list of all the files in the directory that are speeches
NO_OPEN = ['soumeta.txt', 'speeches.json']
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f)) if f not in NO_OPEN]
year_speech = [[f[:-9], int(f[-8:-4]), open(mypath + "/" + f).read()] for f in onlyfiles]

## Q2: Friends, Romans, countrymen, lend me your vectors
Use the classic vector space model from IR to find similar SOU addresses.

### a) Compute the TF-IDF vectors for each SOU address. You should
- lower case all text
- remove punctuation

In [3]:
# Initialize a SparkContext
sc = SparkContext("local", "Simple App")

In [4]:
# Load speeches to SparkContext and sort by year
speeches = sc.parallelize(year_speech).sortBy(lambda tup: tup[1])

In [5]:
# Create RCC with [President, year, speech]
sentences = speeches.map(lambda x: [x[0], x[1], x[2]])

In [6]:
#Define punctuation to strip from documents
PUNCTUATION = '.,:;()-"[]{}' + "'"

In [7]:
# Strip punctuation and lowercase 
words_stripped = sentences.map(lambda x: x[2].lower().translate(None, PUNCTUATION).split())

In [8]:
# This will give us some (sparse) vectors for every document. The vectors are of size 2^16=65,536. This number
# defaults to 2^20 but when trying to do k-means clustering, jupyter runs out of memory. It could even be restricted
# to a smaller size.
hashingTF = HashingTF(numFeatures=65536)
tf = hashingTF.transform(words_stripped)

In [9]:
tf.take(1)

[SparseVector(65536, {83: 1.0, 351: 2.0, 379: 2.0, 553: 1.0, 730: 1.0, 755: 1.0, 929: 1.0, 950: 1.0, 1223: 1.0, 1243: 2.0, 1257: 4.0, 1320: 3.0, 1331: 1.0, 1388: 1.0, 1580: 1.0, 1595: 9.0, 1750: 1.0, 1809: 1.0, 1845: 1.0, 1890: 1.0, 2284: 1.0, 2548: 1.0, 2590: 1.0, 2630: 1.0, 2683: 1.0, 2784: 1.0, 3071: 1.0, 3225: 1.0, 3245: 3.0, 3305: 1.0, 3421: 1.0, 3449: 2.0, 3592: 2.0, 3714: 1.0, 3873: 1.0, 3932: 15.0, 4006: 1.0, 4119: 2.0, 4145: 49.0, 4164: 1.0, 4291: 1.0, 4415: 10.0, 4514: 1.0, 4729: 1.0, 4730: 1.0, 4918: 1.0, 5009: 1.0, 5041: 1.0, 5348: 2.0, 5747: 1.0, 5753: 2.0, 5768: 2.0, 5898: 2.0, 5981: 1.0, 5998: 8.0, 6082: 1.0, 6115: 1.0, 6137: 1.0, 6206: 2.0, 6318: 1.0, 6434: 1.0, 6536: 1.0, 6890: 2.0, 6957: 2.0, 7199: 1.0, 7215: 2.0, 7237: 1.0, 7239: 4.0, 7289: 1.0, 7751: 3.0, 7801: 1.0, 8038: 1.0, 8075: 1.0, 8096: 1.0, 8131: 1.0, 8204: 1.0, 8220: 1.0, 8403: 2.0, 8822: 1.0, 8935: 1.0, 8966: 1.0, 8982: 1.0, 9112: 1.0, 9130: 1.0, 9139: 1.0, 9245: 1.0, 9343: 2.0, 9398: 1.0, 9410: 2.0, 9412:

In [10]:
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)

### (b) Similarity between SOU's
#### 50 most stimilar pairs of SOUs given by different Presidents

In [11]:
def similarity(a, b):
    '''
    Takes:
        a, b, lists with [president, year, tfidf vector]
    Calculates cosine similarity between tfidf vectors of a and b.'''
    return a[2].dot(b[2]) / math.sqrt(a[2].dot(a[2]) * b[2].dot(b[2]))

In [12]:
sou_with_vectors = tfidf.zip(sentences).map(lambda x: [x[1][0], x[1][1], x[0]])

In [13]:
diff_pres = sou_with_vectors.cartesian(sou_with_vectors).filter(lambda x: x[0][0] != x[1][0])

In [14]:
diff_pres_sim = diff_pres.map(lambda pair: [pair[0][0], pair[1][0], pair[0][1], pair[1][1],\
                                      similarity(pair[0], pair[1])]).sortBy(lambda tup: tup[4], ascending=False)

In [15]:
diff_pres_results = diff_pres_sim.collect()
diff_pres_res = [diff_pres_results[i] for i in range(len(diff_pres_results)) if i%2 == 0]
diff_pres_res[:50]

[['William_J._Clinton', 'Barack_Obama', 1995, 2010, 0.45066786316531809],
 ['William_J._Clinton', 'Barack_Obama', 1995, 2012, 0.42389028951035135],
 ['William_J._Clinton', 'Barack_Obama', 1995, 2014, 0.42161066158856031],
 ['William_J._Clinton', 'Barack_Obama', 1993, 2010, 0.41707423779648189],
 ['William_J._Clinton', 'Barack_Obama', 1994, 2010, 0.40624974945861375],
 ['John_Tyler', 'James_K._Polk', 1844, 1846, 0.40439548065401576],
 ['William_J._Clinton', 'Barack_Obama', 1995, 2016, 0.40016317070440804],
 ['William_J._Clinton', 'Barack_Obama', 1995, 2011, 0.39846359218367622],
 ['William_J._Clinton', 'Barack_Obama', 1995, 2015, 0.39402267612637276],
 ['William_J._Clinton', 'Barack_Obama', 1993, 2011, 0.39257039009946326],
 ['Andrew_Jackson', 'Martin_Van_Buren', 1836, 1839, 0.39102844024612982],
 ['William_J._Clinton', 'Barack_Obama', 1994, 2012, 0.39079239535905708],
 ['William_J._Clinton', 'Barack_Obama', 1993, 2012, 0.38936528292622108],
 ['William_J._Clinton', 'Barack_Obama', 1993,

#### 50 most similar pairs of SOUs given by the SAME president

In [16]:
same_pres = sou_with_vectors.cartesian(sou_with_vectors).filter(lambda x: x[0][0] == x[1][0] and x[0][1] != x[1][1])

In [17]:
same_pres_sim = same_pres.map(lambda pair: [pair[0][0], pair[1][0], pair[0][1], pair[1][1],\
                                      similarity(pair[0], pair[1])]).sortBy(lambda tup: tup[4], ascending=False)

In [18]:
same_pres_results = same_pres_sim.collect()
same_pres_res = [same_pres_results[i] for i in range(len(same_pres_results)) if i%2 == 0]

In [19]:
same_pres_res[:50]

[['Barack_Obama', 'Barack_Obama', 2010, 2012, 0.56949171924839703],
 ['Barack_Obama', 'Barack_Obama', 2010, 2011, 0.55794283726692639],
 ['George_W._Bush', 'George_W._Bush', 2007, 2008, 0.53262196504098258],
 ['Barack_Obama', 'Barack_Obama', 2011, 2012, 0.5266175786033126],
 ['Barack_Obama', 'Barack_Obama', 2010, 2014, 0.52552676298505197],
 ['Barack_Obama', 'Barack_Obama', 2014, 2015, 0.5253070827361116],
 ['William_J._Clinton', 'William_J._Clinton', 1999, 2000, 0.52131916078762541],
 ['William_J._Clinton', 'William_J._Clinton', 1998, 1999, 0.51680263498631673],
 ['William_McKinley', 'William_McKinley', 1899, 1900, 0.51056803438083176],
 ['Barack_Obama', 'Barack_Obama', 2012, 2014, 0.50794618875694963],
 ['William_J._Clinton', 'William_J._Clinton', 1997, 1998, 0.50150381892151141],
 ['William_J._Clinton', 'William_J._Clinton', 1998, 2000, 0.50046657727222665],
 ['Barack_Obama', 'Barack_Obama', 2015, 2016, 0.50038638947836089],
 ['Barack_Obama', 'Barack_Obama', 2010, 2015, 0.4966398817

#### 25 most similar pairs of PRESIDENTS, averaging similarity over their SOUs

In [20]:
sum_pres = sc.parallelize(diff_pres_res).map(lambda x: [(x[0], x[1]), (x[4], 1)]).reduceByKey(\
            lambda x, y: [x[0] + y[0], x[1] + y[1]]).map(lambda x: [x[0], x[1][0] / x[1][1]]).sortBy(\
            lambda x: x[1], ascending=False)

In [21]:
sum_pres.take(25)

[[('William_J._Clinton', 'Barack_Obama'), 0.32047043952494458],
 [('Zachary_Taylor', 'Millard_Fillmore'), 0.3091098445897616],
 [('George_Bush', 'William_J._Clinton'), 0.28355830862776432],
 [('James_K._Polk', 'Zachary_Taylor'), 0.28175776742067576],
 [('James_K._Polk', 'Millard_Fillmore'), 0.27863443896053469],
 [('Benjamin_Harrison', 'Grover_Cleveland'), 0.27531318650763553],
 [('Andrew_Jackson', 'Martin_Van_Buren'), 0.27487662304426386],
 [('Theodore_Roosevelt', 'William_Howard_Taft'), 0.26940030231243306],
 [('Rutherford_B._Hayes', 'Chester_A._Arthur'), 0.2693862051761256],
 [('George_Bush', 'Barack_Obama'), 0.26320806458339235],
 [('John_Tyler', 'James_K._Polk'), 0.26162609074712229],
 [('Rutherford_B._Hayes', 'Benjamin_Harrison'), 0.25691746463199117],
 [('Rutherford_B._Hayes', 'Grover_Cleveland'), 0.2541275401483995],
 [('Martin_Van_Buren', 'John_Tyler'), 0.25241087282692637],
 [('James_K._Polk', 'James_Buchanan'), 0.24765476504605999],
 [('Millard_Fillmore', 'Franklin_Pierce'),

Comments on findings:

It's hard to tell if so many speeches are similar by reading them, as they are significantly lengthy documents. Even the online speeches are long (e.g. Clinton's 1995 speech on YouTube is 1h25). Tyler and Polk do speak A LOT about Mexico; which makes sense because of the Mexican-American War that ocurred after Texas annexation to the US in that period. But judging similarity of so many speeches would require several hours to read all of them.

Nevertheless, regarding a better measure of "similarity", one could try:
- Measuring common topics of speeches (even if they don't use the exact same words) using techniques like word2vec.
- Finding similar grammatical patterns using a Naive Bayes model. This would mean training a model with speeches from one president and then calculating how likely it is that another president or speech was produced with the training data. 
- Finally, taking into account sentiment of the speech. 

There might be a way to combine all of these into a single pipeline?



### (c) Cluster the speeches using k-means

In [22]:
some = sou_with_vectors.map(lambda x: x[2])

Let's compute the "cost" (i.e. empirical risk) of each clustering. Let's take then the marginal decrease in risk, to see how much we're improving with each extra cluster.

(Note: Notice how the risk seems to be non-increasing as k increases. Unfortunately, there are times where risk actually goes up. This is due to the initialization of clusters, and the fact that the algorithms returns local minima.)

#### Using random initialization

In [30]:
rand_res = []
for i in range(1,11):
    clusters = KMeans.train(some, i, maxIterations=50, runs=10, seed=1, initializationMode='random')
    cost = clusters.computeCost(some)
    print("k = {0}, cost: {1:,.0f}".format(i, cost))
    rand_res.append(cost)

k = 1, cost: 4,542,241
k = 2, cost: 4,518,591
k = 3, cost: 4,264,491
k = 4, cost: 4,158,019
k = 5, cost: 3,910,926
k = 6, cost: 3,875,895
k = 7, cost: 3,755,000
k = 8, cost: 3,552,139
k = 9, cost: 3,719,476
k = 10, cost: 3,578,208


In [31]:
for x in range(1,len(rand_res)):
    print("k={0}: {1:,.0f}".format(x, rand_res[x]-rand_res[x-1]))

k=1: -23,649
k=2: -254,101
k=3: -106,471
k=4: -247,093
k=5: -35,031
k=6: -120,895
k=7: -202,861
k=8: 167,337
k=9: -141,268


#### Using k-means++ initialization

In [32]:
km_res = []
for i in range(1,11):
    clusters = KMeans.train(some, i, maxIterations=50, runs=10, seed=1, initializationMode='k-means||')
    cost = clusters.computeCost(some)
    print("k = {0}, cost: {1:,.0f}".format(i, cost))
    km_res.append(cost)

k = 1, cost: 4,542,241
k = 2, cost: 4,431,084
k = 3, cost: 4,137,085
k = 4, cost: 3,879,967
k = 5, cost: 3,805,591
k = 6, cost: 3,711,384
k = 7, cost: 3,420,000
k = 8, cost: 3,544,678
k = 9, cost: 3,355,572
k = 10, cost: 3,322,908


In [33]:
for x in range(1,len(km_res)):
    print("k={0}: {1:,.0f}".format(x, km_res[x]-km_res[x-1]))

k=1: -111,157
k=2: -293,998
k=3: -257,119
k=4: -74,376
k=5: -94,207
k=6: -291,384
k=7: 124,678
k=8: -189,105
k=9: -32,665


It would seem that initializing the clustering with k-means++ yields a lower error than random assignment. This might be useful, but we should be careful not to overfit our data.

In [34]:
for i, num in enumerate(np.array(rand_res) - np.array(km_res)):
    print("{0} clusters, difference of {1:,.0f}".format(i, num))

0 clusters, difference of 0
1 clusters, difference of 87,508
2 clusters, difference of 127,405
3 clusters, difference of 278,053
4 clusters, difference of 105,335
5 clusters, difference of 164,512
6 clusters, difference of 335,000
7 clusters, difference of 7,461
8 clusters, difference of 363,904
9 clusters, difference of 255,300


Comments:

The problems with these clusters is that they live in a significantly high-dimensional space, so it's hard to interpret them. We could take the top 20 TF-IDF values of each one and see if we can get any insights on the documents they represent.